In [ ]:
import pybullet as p
import time
import pybullet_data
import os
import numpy as np
import pandas as pd
import math
import random
import sympy as sym

In [ ]:
#ur5 arm specifications

#6 degrees of freedom (6 revolute joints, 6 joint variables)
#Q=(theta_1, theta_2, theta_3, theta_4, theta_5, theta_6)T

dicc = {'link':[i for i in range(1, 7)], 'a_i':[0, -0.425, -0.39225, 0, 0, 0],
        'alpha_i':[math.pi/2, 0, 0, math.pi/2, -math.pi/2, 0],
        'd_i':[0.089159, 0, 0, 0.10915, 0.09465, 0.0823], 
        'theta_i':[sym.Symbol('theta_1'), sym.Symbol('theta_2'), sym.Symbol('theta_3'), 
                   sym.Symbol('theta_4'), sym.Symbol('theta_5'), sym.Symbol('theta_6')]}

df_manipulator_parameters=pd.DataFrame(dicc)
df_manipulator_parameters.set_index(keys="link", inplace=True, drop=True)
display(df_manipulator_parameters)

In [ ]:
def R(angle, axis):
    s=sym.sin(angle)
    c=sym.cos(angle)

    if(axis=='x'):
        R_3x3=sym.Matrix([[1,0,0], [0, c, -s], [0, s, c]])
    elif(axis=='y'):
        R_3x3=sym.Matrix([[c,0,s], [0, 1, 0], [-s, 0, c]])
    elif(axis=='z'):
        R_3x3=sym.Matrix([[c,-s,0], [s, c, 0], [0, 0, 1]])
    
    R = R_3x3.row_join(sym.Matrix([[0], [0], [0]]))
    R = R.col_join(sym.Matrix([[0, 0, 0, 1]]))
    
    return(R)

In [ ]:
def D(d, axis_index):
    D=sym.eye(4)
    D[-1, -1]=1
    D[axis_index, -1]=d

    return(D)
    

In [ ]:
def homogeneous_transform(theta_i, alpha_i, a_i, d_i):
    return D(d_i, 2)@R(theta_i, 'z')@D(a_i, 0)@R(alpha_i, 'x')


In [ ]:
def velocity_prop(j_v_j, j_w_j, j_plus_1_R_j, j_P_j_plus_1, theta_j_plus_1): #j mínima es 0
    #j_Z_j=np.array([[0], [0], [1]])
    j_Z_j = sym.Matrix([[0], [0], [1]])
    d_theta_j_plus_1=sym.Symbol('d_'+str(theta_j_plus_1))
    
    j_plus_1_w_j_plus_1=j_plus_1_R_j@(j_w_j+d_theta_j_plus_1*j_Z_j)
    j_plus_1_v_j_plus_1=j_plus_1_R_j@(j_v_j+sym.Matrix(np.cross((j_w_j+d_theta_j_plus_1*j_Z_j).reshape(1,3), j_P_j_plus_1.reshape(1,3)).reshape(3,1)))
    
    
    return(j_plus_1_w_j_plus_1, j_plus_1_v_j_plus_1)

In [9]:
def direct_kinematics_and_link_velocities():
    dicc_transformadas={}
    
    v_link_vector=np.zeros((3,1)) #0v0 #vector de velocidades lineales de los enlaces
    w_link_vector=np.zeros((3,1)) #0w0 #vector de velocidades angulares de los enlaces
    
    for i in range(1,7): #i mínima es 1
        #print('iteración: ', i)
        theta_i=df_manipulator_parameters.loc[i, 'theta_i']
        alpha_i=df_manipulator_parameters.loc[i, 'alpha_i']
        a_i=df_manipulator_parameters.loc[i, 'a_i']
        d_i=df_manipulator_parameters.loc[i, 'd_i']
        
        i_minus_1_T_i=homogeneous_transform(theta_i, alpha_i, a_i, d_i)
        
        dicc_transformadas[str(i-1)+'T'+str(i)]=i_minus_1_T_i

        #Para el calculo de velocidades de los enlaces, j=i-1
        j_plus_1_R_j = i_minus_1_T_i[0:3, 0:3].reshape(3,3)@sym.Matrix([[1, 0, 0], [0, -1, 0], [0, 0, 1]]) #Inversa de i_minus_1_T_i,
            #Para la primera iteración, se obtiene 1_R_0 de 0_T_1
        j_P_j_plus_1 = i_minus_1_T_i[0:3, 3].reshape(3,1) #Vector de posición
        
        
        w, v=velocity_prop(j_v_j=v_link_vector[0:3,i-1].reshape(3,1), j_w_j=w_link_vector[0:3,i-1].reshape(3,1), 
                           j_plus_1_R_j=j_plus_1_R_j, j_P_j_plus_1=j_P_j_plus_1, theta_j_plus_1=theta_i)
        
        v_link_vector=np.append(v_link_vector, v, axis=1)
        w_link_vector=np.append(w_link_vector, w, axis=1)
        #print('vector velocidades: ', v_link_vector)
        
    T_total=sym.eye(4)
    
    for transformada in reversed(dicc_transformadas.values()):
        T_total=transformada@T_total
        
    
    #Cáculo de las velocidades del efector final desde el sistema {0}
    o_R_ef = T_total[0:3, 0:3].reshape(3,3) #T_total es 0_T_ef
    
    o_v_ef = o_R_ef@v_link_vector[0:3, -1].reshape(3,1) #v_link_vector[0:2, -1] es la velocidad lineal del último eslabón, ef_v_ef
    o_w_ef = o_R_ef@w_link_vector[0:3, -1].reshape(3,1) #w_link_vector[0:2, -1] es la velocidad angular del último eslabón, ef_w_ef  
    
    return(T_total, v_link_vector, w_link_vector, o_v_ef, o_w_ef)

In [10]:
def jacobian(T_total, o_v_ef, o_w_ef):
    T_total = sym.Matrix(T_total)
    dtheta_vector = [sym.Symbol('d_' + str(theta_i)) for theta_i in df_manipulator_parameters['theta_i']] #Vector de velocidades articulares
    
    o_v_ef = sym.Matrix(o_v_ef) #Velocidad lineal del efector final
    o_w_ef = sym.Matrix(o_w_ef) #Velocidad angular del efector final
    velocities = o_v_ef.col_join(o_w_ef) #Vector de velocidades cartesianas del efector final
    
    o_J=sym.zeros(6,6) #Inicialización del jacobiano
    
    #(i_v) = iJ(theta) * d(Q)
    #(i_w)
    
    for i in range(0, len(velocities)):
        for dtheta, j in zip(dtheta_vector, range(0,7)):
            o_J[i,j] = velocities[i].diff(dtheta)
    
    return(o_J)

In [11]:
def inversa_jacob(o_J):
    n=6 #6 grados de libertad
    R=o_J.rank()
    m=6 #Dimensiones del espacio cartesiano
    
    #print(f'n={n}')
    #print(f'R={R}')
    #print(f'm={m}')
    #print(f'o_J.shape={o_J.shape}')
    
    if(n>m and m==R): 
        #Infinitas soluciones, y se escoge la que posee menor norma
        #euclídea mediante la pseudo-inversa a la derecha.
        pseudoinv_o_J = o_J.T@(o_J@o_J.T).inv()
    
    elif(n<m and n==R):
        #No existe solución, y se escoge la que minimiza la expresión
        #||J(θ)Δθ-Δr|| mediante la pseudo-inversa a la izquierda.
        pseudoinv_o_J = (o_J.T@o_J).inv()@o_J
        
    elif(R<n and R<m):
        #Se aplica el algoritmo de Greville.
        
        #ALGORITMO DE GREVILLE
        #---------------------
        
        j1 = o_J[:,0] #Primera columna del jacobiano
        
        if(j1.norm() < 0.001): #Si la primera columna del jacobiano está compuesta de ceros
            a = j1.T #La primera columna de la pseudoinversa está compuesta de ceros
        else:
            a = ((j1.T@j1).inv()@j1.T)
                                #(1,6)(6,1)    (1,6) ==> (1,6)
                                    #(1,1)
            
        for k in range(1,6): #Para el resto de columnas
            #print(f'columna: {k}')
            jk = o_J[:,k].reshape(6,1) #Columna k del jacobiano
            dk = a@jk
                                #(6,k) (6,1)
            ck = jk - o_J[:, :k]@dk
            
            if(ck.norm() < 0.001):
                bk = (sym.Matrix([[1]])+dk.T@dk).inv()@dk.T@a
                
            else:
                bk = (ck.T@ck).inv()@ck.T
            
            a=(a-dk@bk).col_join(bk)
            
            #print(f'a: {a}')
    
    else: 
         a=(o_J.T).adjugate() / o_J.det()
                
    return(a)


In [12]:
def diff_kinematics(theta_vector_o, target_pos):
    #Formas generales de la transformada global, velocidades y jacobiano
    T_total, v_link_vector, w_link_vector, o_v_ef, o_w_ef = direct_kinematics_and_link_velocities()
    o_J = jacobian(T_total, o_v_ef, o_w_ef)
    
    epsilon=0.001
    incr_theta=sym.ones(6,1) #Valor mayor que épsilon
    theta_vector=theta_vector_o
    
    theta_symbols = [sym.Symbol(f'theta_{i}') for i in range(1, 7)]
    
    while(incr_theta.norm()>epsilon): 
        #Mientras el incremento en el vector theta sea meno que épsilon todavía no se
        #ha alcanzado el objetivo 
        
        o_J_evaluado=o_J
        T_total_evaluado=T_total
    
        o_J_evaluado=o_J_evaluado.evalf(subs={theta_symbols[i]: float(theta_vector[i]) for i in range(6)})
        #print('o_J_evaluado')
        #print(o_J_evaluado)
        
        
        T_total_evaluado=T_total_evaluado.evalf(subs={theta_symbols[i]: float(theta_vector[i]) for i in range(6)})
        #print('T_total_evaluado')
        #print(T_total_evaluado)
        
        #Euler angles:
        Y = math.asin(-T_total_evaluado[2,0])
        
        if abs(Y)!=math.pi/2:
            Z=math.atan2(T_total_evaluado[1, 0], T_total_evaluado[0, 0])
            X=math.atan2(T_total_evaluado[2, 1], T_total_evaluado[2, 2])
            
        else: #Gimbal lock
            Z=math.atan2(-T_total_evaluado[0, 1], T_total_evaluado[1, 1])
            X=0
            
        print(X, Y, Z)
        
        pos_new=T_total_evaluado[0:3, -1].col_join(sym.Matrix([[X], [Y], [Z]]))
        incr_pos = target_pos-pos_new
        incr_theta = inversa_jacob(o_J_evaluado)@incr_pos
        print('Incremento theta: '+str(incr_theta))
        theta_vector=theta_vector+incr_theta
        
    return(theta_vector)

## MAIN LOOP

In [ ]:

theta_vector_o=sym.zeros(6,1)
target_pos=sym.Matrix([[0.5], [0.0], [0.5], [0.0], [0.0], [0.0]]) #[X, Y, Z, EulerX, EulerY, EulerZ]
    
diff_kinematics(theta_vector_o, target_pos)
